# Load Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import imshow

import h5py
import cv2

slim = tf.contrib.slim

#keras libraries
import keras
from keras import layers
from keras.layers import Input, Dropout, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model, load_model
#from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.initializers import glorot_uniform

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from sklearn.preprocessing import LabelBinarizer
import random
from random import randint #to generate random numbers
import pydot

# Read The H5 Valid Data

In [ ]:
hdf5_dataset = {}
hdf5_dataset_list = ['batch', 'label', 'hotlabel', 'img_name']
hdf5_dataset['name'] = hdf5_dataset_list

In [ ]:
def get_hdf5(hdf5_dataset, hdf5_path):
       
    # val-test
    # your val set labels
    with h5py.File(hdf5_path,'r') as hdf5_file:
       
        # Total number of samples
        X_batch = np.array(hdf5_file[hdf5_dataset['name'][0]][:])

        #Y_label = np.array(hdf5_file[hdf5_dataset['name'][1]][:])
        Y_hot_label = np.array(hdf5_file[hdf5_dataset['name'][2]][:])
        img_name = np.array(hdf5_file[hdf5_dataset['name'][3]][:])
    
        return X_batch, Y_hot_label, img_name
    

## Identity Block

In [ ]:
# identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut]) 
    X = Activation('relu')(X)

    return X

## Conv Block

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (3, 3), strides = (s,s), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1,1), padding = 'same', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(F3, (3, 3), strides = (s,s), padding = 'same', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# Build Model

In [ ]:
def ResNet50(input_shape = (300, 300, 3), classes = 7):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((1, 1))(X_input)

    # Stage 1
    X = Conv2D(16, (3, 3), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    print('Conv1a: ' + str(X.shape))
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    
    # Stage 2
    X = convolutional_block(X, f = 3, filters = [16, 16, 32], stage = 2, block='a', s = 2)
    print('Convblock2a: ' + str(X.shape))
    X = identity_block(X, 3, [32, 32, 32], stage=2, block='b')
    print('Identityblock3b: ' + str(X.shape))
    
    X = MaxPooling2D( strides=(2, 2))(X)
    print('MaxPool2a: ' + str(X.shape))

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [32, 32, 64], stage = 3, block='a', s = 2)
    print('Convblock3a: ' + str(X.shape))
    X = identity_block(X, 3, [64, 64, 64], stage=3, block='b')
    print('Identityblock3b: ' + str(X.shape))
    X = identity_block(X, 3, [64, 64, 64], stage=3, block='c')
    print('Identityblock3c: ' + str(X.shape))

    X = MaxPooling2D( strides=(2, 2))(X)
    print('MaxPool3a: ' + str(X.shape))
    
    # Stage 4
    X = convolutional_block(X, f = 3, filters = [64, 64, 128], stage = 4, block='a', s = 2)
    print('Convblock4a: ' + str(X.shape))
    X = identity_block(X, 3, [128, 128, 128], stage=4, block='b')
    print('Identityblock4b: ' + str(X.shape))

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [128, 128, 256], stage = 5, block='a', s = 2)
    print('Convblock5a: ' + str(X.shape))

    # MAXPOOL 
    X = MaxPooling2D( strides=(2, 2))(X)
    print('MaxPool: ' + str(X.shape))

    # output layer
    X = Flatten()(X)
    
    X = Dense(64)(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)
    
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
# Run the model
model = ResNet50(input_shape = (300, 300, 3), classes = 7)

In [ ]:
#def top_1_accuracy(X_train, Y_train):
#    return keras.metrics.top_k_categorical_accuracy(X_train, Y_train, k=1)
#
#def top_2_accuracy(X_train, Y_train):    
#    return keras.metrics.top_k_categorical_accuracy(X_train, Y_train, k=2)
#
#def top_3_accuracy(X_train, Y_train):
#    return keras.metrics.top_k_categorical_accuracy(X_train, Y_train, k=3)

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[top_1_accuracy, top_2_accuracy, top_3_accuracy])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training Part

In [ ]:
# Load the Validation Data
hdf5_path = 'read/it/from/this/path/val.hdf5'
X_val, Y_hot_label_val, img_name_val = get_hdf5(hdf5_dataset, hdf5_path)

In [ ]:
history = {}

In [ ]:
def train(hyperparameters, hdf5_base_path, history):

    history['loss'] = []
    history['acc'] = []
    history['val_loss'] = []
    history['val_acc'] = []
    
    for i in range(1, hyperparameters['epoch'] + 1):
        print('epoch: ' + str(i))   
        for k in range(1, hyperparameters['train_batch_len'] + 1):
            print('batch: ' + str(k))   
            # train1
            hdf5_path_train = hdf5_base_path + 'train' + str(k) + '.hdf5'

            with h5py.File(hdf5_path_train,'r') as train_dataset:

                X_train = train_dataset[hdf5_dataset['name'][0]][:]

                Y_hot_label_train = np.array(train_dataset[hdf5_dataset['name'][2]][:])

                model_history = model.fit(X_train, Y_hot_label_train, \
                          epochs = 1, batch_size = hyperparameters['batch_size'], \
                        verbose=1, validation_data = (X_val, Y_hot_label_val))

                history['loss'].append(model_history.history['loss'][0])
                history['acc'].append(model_history.history['acc'][0])
                
                history['val_loss'].append(model_history.history['val_loss'][0])
                history['val_acc'].append(model_history.history['val_acc'][0])
            
    return history

In [ ]:
hyperparameters = {}
hyperparameters['epoch'] = 3
hyperparameters['train_batch_len'] = 32
hyperparameters['batch_size'] = 32

augmentation_list = ['orig', 'orig_flip', 'orig_zoom', 'orig_flip_zoom', 'orig_blur', \
                     'orig_flip_blur', 'orig_noise', 'orig_flip_noise']
hyperparameters['augmentation_list'] = augmentation_list

hdf5_base_path = 'base/path/'

m = (0, 0, 0)
s = (1, 1, 1)
hyperparameters['gause_noise_m'] = m
hyperparameters['gause_noise_s'] = s

save_img_size = (300, 300, 3)
hyperparameters['save_img_size'] = save_img_size

class_number = 7
hyperparameters['class_number'] = class_number

hdf5_dataset_list = ['batch', 'label', 'hotlabel', 'img_name']
hyperparameters['name_dataset'] = hdf5_dataset_list

augmentation_list = ['orig', 'orig_flip', 'orig_zoom', 'orig_flip_zoom', 'orig_blur', \
                     'orig_flip_blur', 'orig_noise', 'orig_flip_noise']
hyperparameters['augmentation_list'] = augmentation_list

### Callbacks 

In [ ]:
history = train(hyperparameters, hdf5_base_path, history)

# Save Model

In [ ]:
model.summary()

In [ ]:
# Saving whole models (architecture + weights + optimizer state)
model.save('save/model/model1.h5')

In [ ]:
# Saving only a model's weights
model.save_weights('save/model/model1_weights.h5')

# Validation

In [ ]:
hdf5_path = 'read/it/from/here/val.hdf5'
X_val, Y_hot_label_val, img_name_val = get_hdf5(hdf5_dataset, hdf5_path)

print ("number of validation examples = " + str(X_val.shape[0]))
print ("X_val shape: " + str(X_val.shape))
print ("Y_val shape: " + str(Y_hot_label_val.shape))

In [ ]:
loss, acc = model.evaluate(X_val, Y_hot_label_val)
print ("Loss = " + str(loss))

print('Top-1 Accuracy {:.1%}'.format(acc))

## Load the Model and Weights

In [ ]:
model = load_model('load/it/from/here/model1.h5')

# Test Model

In [ ]:
hdf5_path = 'read/it/from/here/test.hdf5'
X_test, Y_hot_label_test, img_name_test = get_hdf5(hdf5_dataset, hdf5_path)

print ("number of validation examples = " + str(X_test.shape[0]))
print ("X_val shape: " + str(X_test.shape))
print ("Y_val shape: " + str(Y_hot_label_test.shape))

In [ ]:
loss, acc = model.evaluate(X_test, Y_hot_label_test)
print ("Loss = " + str(loss))

print('Top-1 Accuracy {:.1%}'.format(acc))

## Show the Results

In [ ]:
def display_image_predictions(features, labels, predictions, hyperparameters):
    n_classes = 7
    label_names = ['VW-Passat', 'RENAULT-Fluence', 'FIAT-Linea', 'VW-Polo', 'RENAULT-Toros', 'FIAT-Dogan', 'OtherClass']
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(n_classes))
    label_ids = label_binarizer.inverse_transform(np.array(labels))

    # Softmax Predictions
    fig, axies = plt.subplots(nrows=4, ncols=2, figsize=(20, 18))

    fig.tight_layout()

    n_predictions = hyperparameters['top_n_predictions']
    margin = 0.05
    ind = np.arange(n_predictions)
    width = (1. - 2. * margin) / n_predictions

    for image_i, (feature, label_id, pred_values) in enumerate(zip(features, label_ids, predictions)):
        pred_names = label_names[np.argmax(pred_values)]
        correct_name = 'Sample:'+ str(image_i + 1) + '   |   Correct: ' + label_names[label_id]

        axies[image_i][0].imshow(feature)
        axies[image_i][0].set_title(correct_name, fontsize=14, fontweight='bold')
        axies[image_i][0].set_axis_off()

        axies[image_i][1].barh(ind + margin, pred_values[::-1], width)
        
        axies[image_i][1].set_yticks(ind + margin)
        axies[image_i][1].set_yticklabels(['OtherClass', 'FIAT-Dogan', 'RENAULT-Toros', 'VW-Polo', 'FIAT-Linea', 'RENAULT-Fluence', 'VW-Passat'],fontsize=14, fontweight='bold')
        
        axies[image_i][1].set_xticks([0, 0.5, 0.8, 1.0])
        
    fig.savefig('example.png', bbox_inches='tight')

In [ ]:
hyperparameters['n_samples'] = 4
hyperparameters['top_n_predictions'] = Y_hot_label_test.shape[1]

def show_results(X_test, Y_hot_label_test, hyperparameters):
        
    # Print Random Samples
    random_test_features, random_test_labels = zip(*random.sample(list(zip(X_test, Y_hot_label_test)), \
                                                                  hyperparameters['n_samples']))
    random_test_features = np.reshape(random_test_features, \
                                      (len(random_test_features), hyperparameters['save_img_size'][0],\
                                       hyperparameters['save_img_size'][1], hyperparameters['save_img_size'][2]))
    y_pred = model.predict(random_test_features)
    #print(random_test_labels)
    #print(np.argmax(y_pred, axis =1))
    display_image_predictions(random_test_features, random_test_labels, y_pred, hyperparameters)


show_results(X_test, Y_hot_label_test, hyperparameters)